In [4]:
import json
import numpy as np
import torch
from PIL import Image
from torchvision import models
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Define paths directly
IMAGE_PATH = '../data/test/AI.jpeg'  # Specify the full path to your image
MODEL_PATH = '../VGG16_img224x224_batch32_epochs40_lr0.00010.pth'  # Specify the full path to your model weights
LABELS_JSON_PATH = '../outputs/models/best/label_mapping.json'  # Specify the full path to your labels JSON

# Fixed number of classes
NUM_CLASSES = 3

def create_transforms(image_size):
    """Create and return transformations for the input image."""
    return A.Compose([
        A.Resize(height=image_size[0], width=image_size[1]),
        A.Normalize(mean=(0, 0, 0), std=(1, 1, 1), max_pixel_value=255.0),
        ToTensorV2()
    ])

def load_labels(json_path):
    """Load class labels from a JSON file."""
    with open(json_path, 'r') as f:
        return json.load(f)['labels']

def load_image(image_path, transforms):
    """Load and preprocess the image for VGG16 using Albumentations."""
    image = Image.open(image_path).convert("RGB")
    image = np.array(image)  # Convert to numpy array for Albumentations
    transformed_image = transforms(image=image)['image']  # Apply transformations
    return transformed_image.unsqueeze(0)  # Add a batch dimension

def get_vgg16_model(model_weights_path):
    """Initialize and return the VGG16 model with specified weights."""
    model = models.vgg16(weights=None)  # Set weights=None to prevent downloading
    model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, NUM_CLASSES)

    # Load your custom weights
    state_dict = torch.load(model_weights_path, map_location='cpu')

    # Remove 'module.' prefix if it exists
    if 'module.' in next(iter(state_dict.keys())):
        state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}

    model.load_state_dict(state_dict)
    model.eval()  # Set the model to evaluation mode
    return model


def predict(image_path, labels_json_path):
    """Load image, model, and make predictions."""
    labels = load_labels(labels_json_path)
    
    # Create transforms with the desired image size (224x224 for VGG16)
    transforms = create_transforms(image_size=(224, 224))
    
    # Load and preprocess the image
    image = load_image(image_path, transforms)

    model = get_vgg16_model(MODEL_PATH)

    with torch.no_grad():
        logits = model(image)  # Raw predictions
        probabilities = torch.nn.functional.softmax(logits, dim=1)  # Convert to probabilities
        predicted_class_index = torch.argmax(probabilities, dim=1).item()  # Get the predicted class index

    predicted_label = labels[str(predicted_class_index)]
    return predicted_label, predicted_class_index, probabilities

if __name__ == "__main__":
    predicted_label, predicted_class_index, probabilities = predict(IMAGE_PATH, LABELS_JSON_PATH)

    print("Predicted class index:", predicted_class_index)
    print("Predicted label:", predicted_label)
    print("Probabilities:", probabilities)


/tmp/ipykernel_56382/2533136695.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_weights_path, map_location='cpu')


Predicted class index: 0
Predicted label: AI_cohort
Probabilities: tensor([[9.9980e-01, 2.2191e-08, 1.9826e-04]])


In [6]:
import json
import numpy as np
import torch
from PIL import Image
from torchvision import models
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Define paths directly
IMAGE_PATH = '../data/test/PXL_20241025_201012979.jpg'  # Specify the full path to your image
MODEL_PATH = '../outputs/models/best/_epoch34_best.pth'  # Specify the full path to your model weights
LABELS_JSON_PATH = '../outputs/models/best/label_mapping.json'  # Specify the full path to your labels JSON

# Fixed number of classes
NUM_CLASSES = 3

def create_transforms(image_size):
    """Create and return transformations for the input image."""
    return A.Compose([
        A.Resize(height=image_size[0], width=image_size[1]),
        A.Normalize(mean=(0, 0, 0), std=(1, 1, 1), max_pixel_value=255.0),
        ToTensorV2()
    ])

def load_labels(json_path):
    """Load class labels from a JSON file."""
    with open(json_path, 'r') as f:
        return json.load(f)['labels']

def load_image(image_path, transforms):
    """Load and preprocess the image for VGG16 using Albumentations."""
    image = Image.open(image_path).convert("RGB")
    image = np.array(image)  # Convert to numpy array for Albumentations
    transformed_image = transforms(image=image)['image']  # Apply transformations
    return transformed_image.unsqueeze(0)  # Add a batch dimension

def get_vgg16_model(model_weights_path):
    """Initialize and return the VGG16 model with specified weights."""
    model = models.vgg16(weights=None)  # Set weights=None to prevent downloading
    model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, NUM_CLASSES)

    # Load your custom weights
    checkpoint = torch.load(model_weights_path, map_location='cpu')
    state_dict = checkpoint['model_state_dict']  # Extract model state dict from checkpoint

    # Remove 'module.' prefix if it exists
    if 'module.' in next(iter(state_dict.keys())):
        state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}

    model.load_state_dict(state_dict)
    model.eval()  # Set the model to evaluation mode
    return model


def predict(image_path, labels_json_path):
    """Load image, model, and make predictions."""
    labels = load_labels(labels_json_path)
    
    # Create transforms with the desired image size (224x224 for VGG16)
    transforms = create_transforms(image_size=(224, 224))
    
    # Load and preprocess the image
    image = load_image(image_path, transforms)

    model = get_vgg16_model(MODEL_PATH)

    with torch.no_grad():
        logits = model(image)  # Raw predictions
        probabilities = torch.nn.functional.softmax(logits, dim=1)  # Convert to probabilities
        predicted_class_index = torch.argmax(probabilities, dim=1).item()  # Get the predicted class index

    predicted_label = labels[str(predicted_class_index)]
    return predicted_label, predicted_class_index, probabilities

if __name__ == "__main__":
    predicted_label, predicted_class_index, probabilities = predict(IMAGE_PATH, LABELS_JSON_PATH)

    print("Predicted class index:", predicted_class_index)
    print("Predicted label:", predicted_label)
    print("Probabilities:", probabilities)


/tmp/ipykernel_8485/561143928.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_weights_path, map_location='cpu')


Predicted class index: 0
Predicted label: AI_cohort
Probabilities: tensor([[9.9929e-01, 1.6859e-05, 6.9129e-04]])


# next
get model weights only from 
optimize speed 
qunatize model
deploy model
